# Run builtwith on sites

In [1]:
import pandas as pd
import requests
import json 
import re
import os

#Make sure there's somewhere to put the data
if not os.path.exists('data/builtwith'):
    os.makedirs('data/builtwith')

In [2]:
#fromsites = ['http://example.com']
fromsites = ['https://www.txfreedomforce.org/']

f = open('/Users/sara/builtwithkey.txt', 'r')
bwkey = f.readline().strip()
f.close()
bwapi = 'rv1' # 'free1' is the free api

allmatches = pd.DataFrame([])
allidentifiers = pd.DataFrame([])

for bwdom in fromsites:
    print(bwdom)
    try:
        bwurl = 'https://api.builtwith.com/{}/api.json?KEY={}&LOOKUP={}'.format(bwapi, bwkey, bwdom)
        bwresp = requests.get(bwurl)

        with open('data/builtwith/{}.json'.format(bwdom), 'w') as outfile:
            json.dump(bwresp.json(), outfile)

        matches = pd.DataFrame([])
        identifiers = pd.DataFrame([])

        rs = bwresp.json()['Relationships']
        for thisr in rs:
            fromdomain = thisr['Domain']
            rsi = thisr['Identifiers']

            ids = pd.DataFrame(rsi).drop('Matches', axis=1)
            ids['FromDomain'] = fromdomain
            identifiers = identifiers.append(ids)

            for rsix in rsi:
                rsimatches = pd.DataFrame(rsix['Matches'])
                rsimatches['Type'] = rsix['Type']
                rsimatches['Value'] = rsix['Value']
                rsimatches['FromDomain'] = fromdomain
                if len(rsimatches) > 0:
                    matches = matches.append(rsimatches)
        matches.to_csv('data/builtwith/{}_matches.csv'.format(bwdom), index=False)
        identifiers.to_csv('data/builtwith/{}_identifiers.csv'.format(bwdom), index=False)
        if len(matches) > 0:
            allmatches = allmatches.append(matches)
        if len(identifiers) > 0:
            allidentifiers = allidentifiers.append(identifiers)
    except:
        continue

allmatches

https://www.txfreedomforce.org/


""


In [3]:
allidentifiers

""
